In [ ]:
from joblib import dump
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_data_raw = pd.read_csv('../data/raw/beer_reviews.csv')

In [ ]:
df_data_clean1 = df_data_raw.copy()

In [ ]:
#Define columns to be used for various steps later
col_usable = ['brewery_name','review_aroma','review_appearance','review_palate','review_taste','beer_abv','beer_style']
independent_cols = ['brewery_name','review_aroma','review_appearance','review_palate','review_taste','beer_abv']
numerical_cols = ['review_aroma','review_appearance','review_palate','review_taste','beer_abv']
factor_cols = ['brewery_name']
target_col = ['beer_style']

In [ ]:
#Select only the required columns
df_data_reduced1 = df_data_clean1.loc[:,col_usable]

In [ ]:
#Save the target column transformer for inverse transformation in app
dump(target_encoder,'../models/target_decoder.joblib')

In [ ]:
cat_var_transformer = Pipeline(
    steps=[
        ('brewery_name_encoder', MinMaxScaler())
    ]
)

num_var_transformer = Pipeline(
    steps=[
        ('beer_measures_encoder', OrdinalEncoder())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('fac_cols', cat_var_transformer, factor_cols),
        ('num_cols', num_var_transformer, numerical_cols)
    ]
)

model_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('log_regression', LogisticRegression(max_iter=10000)) ## This line is set to the final model used for API
    ]
)

In [ ]:
#Use the original sub-dataset for training
df_data_reduced1=df_data_reduced1.dropna()
df_data_reduced1.pop('beer_style')
X1, X2, y1, y2 = train_test_split(df_data_reduced1, target_out, train_size=0.7, random_state=42)
model_pipeline.fit(X1,y1)

In [ ]:
dump(model_pipeline,'../models/pipeline.joblib')